# Filtering Networks
***Kozo Nishida, Kristina Hanspers, Alexander Pico, Barry Demchak***

**Package**

py4cytoscape 0.0.9

This vignette will introduce you to some techniques for filtering a network based on node properties. You will learn to:

- Select a set of nodes based on node degree and attribute filters
- Create a subnetwork based on selected nodes
- Hide a set of nodes based on filters

For this tutorial, we will use data from the STRING database (https://string-db.org/).

---
## Setup data files, py4cytoscape and Cytoscape connection
---
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

---
### Setup: Fetch latest py4cytoscape

**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.10').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@0.0.10' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

---
## Setup: Sanity test to verify Cytoscape connection


By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [ ]:
p4c.cytoscape_version_info()

## Prerequisites

In [ ]:
p4c.install_app('STRINGapp')

## Get network from STRING
We are going to query the STRING Disease database for the term “breast cancer”. By default, the app pulls the top 100 human proteins associated with the disease along with edges having an evidence strength of 0.4 or greater:

In [ ]:
string_cmd = 'string disease query disease="breast cancer"'
p4c.commands_run(string_cmd)
string_net = p4c.get_network_suid()

In [ ]:
from IPython import display
p4c.export_image("img1.png")
p4c.sandbox_get_from("img1.png")
display.Image("img1.png")

## Filtering by degree
### Creating a degree filter
Every node in a network has a Degree property, which corresponds to the number of edges connecting the node to other nodes, either as a target or source. Filtering based on node degree is a useful way to remove nodes with too few (or too many) connections.

In this example we want to exclude low degree nodes, e.g., those with only 0, 1 or 2 connections:


In [ ]:
p4c.create_degree_filter('degree filter', [0,2], 'IS_NOT_BETWEEN')

In [ ]:
p4c.export_image("img2.png")
p4c.sandbox_get_from("img2.png")
display.Image("img2.png")

At the bottom of the Select tab, you can see how many edges/nodes where selected.

### Creating a subnetwork from a selection

We can now create a new network, or subnetwork, from our selected set of nodes and all relevant edges:

In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: highly connected nodes')

In [ ]:
p4c.export_image("img3.png")
p4c.sandbox_get_from("img3.png")
display.Image("img3.png")

## Filtering by attribute

### Creating a column filter

We could also filter the network based on high disease score. The disease score comes from STRING and indicates the strength of the association to the disease queried.

Let’s select nodes from the original network with a disease score of greater than 4 (on a scale of 1-5):

In [ ]:
p4c.create_column_filter(filter_name='disease score filter', column='stringdb::disease score', criterion=4, predicate='GREATER_THAN', network=string_net)

In [ ]:
p4c.export_image("img4.png")
p4c.sandbox_get_from("img4.png")
display.Image("img4.png")

Again, we can create a subnetwork from the selection:

In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: high disease score')

In [ ]:
p4c.export_image("img5.png")
p4c.sandbox_get_from("img5.png")
display.Image("img5.png")

## Combining filters
But what if we want to combine these two filters? You could apply them sequentially as individual filters, but then you’d need to be careful about the order in which you apply the filters. Alternatively, you can create a composite filter and apply the logic all at once!

Let’s combine the two filters “degree filter” and “disease score” to produce one filter, then apply it to the original network and create a final subnetwork:

In [ ]:
p4c.create_composite_filter('combined filter', ['degree filter','disease score filter'], network=string_net)

In [ ]:
p4c.create_subnetwork(subnetwork_name='final subnetwork')

In [ ]:
p4c.export_image("img6.png")
p4c.sandbox_get_from("img6.png")
display.Image("img6.png")

We can apply a layout to help with interpreting the network:

In [ ]:
p4c.layout_network('force-directed defaultSpringCoefficient=5E-6')

In [ ]:
p4c.export_image("img7.png")
p4c.sandbox_get_from("img7.png")
display.Image("img7.png")

This final network obviously contains fewer nodes than the original, but they are the most connected and most highly associated with the disease. If you examine the network you can see several well-known breast cancer oncogenes, for example BRCA1, TP53 and PTEN, near the center of the action.

## Hiding filtered nodes

As a final example of the filter functions, let’s return to the orignal network once more and apply our “combined filter”. But this time let’s *hide* the filtered out nodes, rather than forming a selection. This demonstrates the *applyFilter* function and the *hide* parameter that is optional for all *createXXXFilter* functions as well.

In [ ]:
p4c.apply_filter('combined filter', hide=True, network=string_net)